KWIKVAL - Rui Cruzeiro - Dezembro 2022

O objectivo deste documento é a criação de uma ferramenta simples de avaliação imobiliária (Kwikval) com recurso a Machine Learning. Os passos dados pela ferramenta são os seguintes:
1. INPUTS: Pedir ao utilizador um distrito, um concelho, um tipo de imóvel e a sua Área Bruta Dependente (conforme registado na Caderneta Predial);
2. SCRAPING: Pesquisar os anúncios do Casa Sapo e escolher os que mais se assemelham ao imóvel dado pelo utilizador;
3. HOMOGENEIZAÇÃO: Fazer homogeneização de áreas e dedução da margem de negociação;
4. MACHINE LEARNING: Aplicação de Regressão Linear com o Scikit-Learn (o que corresponde à aplicação do Método Comparativo de Mercado (método tradicional de avaliação imobiliária).

## INPUTS

O utilizador escolhe um distrito e um concelho. Estes serão usados para construir o url do Casa Sapo a usar no request. Para isso, foi construído dicionário com os concelhos para cada distrito, com informação obtida na Internet.

In [10]:
# Dicionário com lista dos concelhos por cada distrito

concelhos_por_distrito = {
    'Açores': [
                'Angra do Heroísmo',
                'Calheta',
                'Corvo',
                'Horta',
                'Lagoa',
                'Lajes das Flores',
                'Lajes do Pico',
                'Madalena',
                'Nordeste',
                'Ponta Delgada',
                'Povoação',
                'Praia da Vitória',
                'Ribeira Grande',
                'Santa Cruz da Graciosa',
                'Santa Cruz das Flores',
                'São Roque do Pico',
                'Velas',
                'Vila do Porto',
                'Vila Franca do Campo'
                ],
    'Aveiro': [
                'Águeda',
                'Albergaria-a-Velha',
                'Anadia',
                'Arouca',
                'Aveiro',
                'Castelo de Paiva',
                'Espinho',
                'Estarreja',
                'Ílhavo',
                'Mealhada',
                'Murtosa',
                'Oliveira de Azeméis',
                'Oliveira do Bairro',
                'Ovar',
                'Santa Maria da Feira',
                'São João da Madeira',
                'Sever do Vouga',
                'Vagos',
                'Vale de Cambra'
                ],
    'Beja': [
                'Aljustrel',
                'Almodôvar',
                'Alvito',
                'Barrancos',
                'Beja',
                'Castro Verde',
                'Cuba',
                'Ferreira do Alentejo',
                'Mértola',
                'Moura',
                'Odemira',
                'Ourique',
                'Serpa',
                'Vidigueira'
                ],
    'Braga': [
                'Amares',
                'Barcelos',
                'Braga',
                'Cabeceiras de Basto',
                'Celorico de Basto',
                'Esposende',
                'Fafe',
                'Guimarães',
                'Póvoa de Lanhoso',
                'Terras de Bouro',
                'Vieira do Minho',
                'Vila Nova de Famalicão',
                'Vila Verde',
                'Vizela'
                ],    
    'Bragança': [
                'Alfândega da Fé',
                'Bragança',
                'Carrazeda de Ansiães',
                'Freixo de Espada à Cinta',
                'Macedo de Cavaleiros',
                'Miranda do Douro',
                'Mirandela',
                'Mogadouro',
                'Torre de Moncorvo',
                'Vila Flor',
                'Vimioso',
                'Vinhais'
                ],
    'Castelo Branco': [
                'Belmonte',
                'Castelo Branco',
                'Covilhã',
                'Fundão',
                'Idanha-a-Nova',
                'Oleiros',
                'Penamacor',
                'Proença-a-Nova',
                'Sertã',
                'Vila de Rei',
                'Vila Velha de Ródão'       
                ],
    'Coimbra': [
                'Arganil',
                'Cantanhede',
                'Coimbra',
                'Condeixa-a-Nova',
                'Figueira da Foz',
                'Góis',
                'Lousã',
                'Mira',
                'Miranda do Corvo',
                'Montemor-o-Velho',
                'Oliveira do Hospital',
                'Pampilhosa da Serra',
                'Penacova',
                'Penela',
                'Soure',
                'Tábua',
                'Vila Nova de Poiares'
                ],
    'Évora': [
                'Alandroal',
                'Arraiolos',
                'Borba',
                'Estremoz',
                'Évora',
                'Montemor-o-Novo',
                'Mora',
                'Mourão',
                'Portel',
                'Redondo',
                'Reguengos de Monsaraz',
                'Vendas Novas',
                'Viana do Alentejo',
                'Vila Viçosa'
                ],
    'Faro': [
                'Albufeira',
                'Alcoutim',
                'Aljezur',
                'Castro Marim',
                'Faro',
                'Lagoa',
                'Lagos',
                'Loulé',
                'Monchique',
                'Olhão',
                'Portimão',
                'São Brás de Alportel',
                'Silves',
                'Tavira',
                'Vila do Bispo',
                'Vila Real de Santo António'
                ],
    'Guarda': [
                'Aguiar da Beira',
                'Almeida',
                'Celorico da Beira',
                'Figueira de Castelo Rodrigo',
                'Fornos de Algodres',
                'Gouveia',
                'Guarda',
                'Manteigas',
                'Mêda',
                'Pinhel',
                'Sabugal'
                'Seia',
                'Trancoso',
                'Vila Nova de Foz Côa'
                ],
    'Leiria': [
                'Alcobaça',
                'Alvaiázere',
                'Ansião',
                'Batalha',
                'Bombarral',
                'Caldas da Rainha',
                'Castanheira de Pera',
                'Figueiró dos Vinhos',
                'Leiria',
                'Marinha Grande',
                'Nazaré',
                'Óbidos',
                'Pedrógão Grande',
                'Peniche',
                'Pombal',
                'Porto de Mós'
                ],
    'Lisboa': [
                'Alenquer',
                'Amadora',
                'Arruda dos Vinhos',
                'Azambuja',
                'Cadaval',
                'Cascais',
                'Lisboa',
                'Loures',
                'Lourinhã',
                'Mafra',
                'Odivelas',
                'Oeiras',
                'Sintra',
                'Sobral de Monte Agraço',
                'Torres Vedras',
                'Vila Franca de Xira'
                ],
    'Madeira': [
                'Calheta',
                'Câmara de Lobos',
                'Funchal',
                'Machico',
                'Ponta do Sol',
                'Porto Moniz',
                'Porto Santo',
                'Ribeira Brava',
                'Santa Cruz',
                'Santana',
                'São Vicente'
                ],
    'Portalegre': [
                'Alter do Chão',
                'Arronches',
                'Avis',
                'Campo Maior',
                'Castelo de Vide',
                'Crato',
                'Elvas',
                'Fronteira',
                'Gavião',
                'Marvão',
                'Monforte',
                'Nisa',
                'Ponte de Sor',
                'Portalegre',
                'Sousel'
                ],
    'Porto': [
                'Amarante',
                'Baião',
                'Felgueiras',
                'Gondomar',
                'Lousada',
                'Maia',
                'Marco de Canaveses',
                'Matosinhos',
                'Paços de Ferreira',
                'Paredes',
                'Penafiel',
                'Porto',
                'Póvoa de Varzim',
                'Santo Tirso',
                'Trofa',
                'Valongo',
                'Vila do Conde',
                'Vila Nova de Gaia'
                ],
    'Santarém': [
                'Abrantes',
                'Alcanena',
                'Almeirim',
                'Alpiarça',
                'Benavente',
                'Cartaxo',
                'Chamusca',
                'Constância',
                'Coruche',
                'Entroncamento',
                'Ferreira do Zêzere',
                'Golegã',
                'Mação',
                'Ourém',
                'Rio Maior',
                'Salvaterra de Magos',
                'Santarém',
                'Sardoal',
                'Tomar',
                'Torres Novas',
                'Vila Nova da Barquinha'
                ],
    'Setúbal': [
                'Alcácer do Sal',
                'Alcochete',
                'Almada',
                'Barreiro',
                'Grândola',
                'Moita',
                'Montijo',
                'Palmela',
                'Santiago do Cacém',
                'Seixal',
                'Sesimbra',
                'Setúbal',
                'Sines'
                ],
    'Viana do Castelo': [
                'Arcos de Valdevez',
                'Caminha',
                'Melgaço',
                'Monção',
                'Paredes de Coura',
                'Ponte da Barca',
                'Ponte de Lima',
                'Valença',
                'Viana do Castelo',
                'Vila Nova de Cerveira     '   
                ],
    'Vila Real': [
                'Alijó',
                'Boticas',
                'Chaves',
                'Mesão Frio',
                'Mondim de Basto',
                'Montalegre',
                'Murça',
                'Peso da Régua',
                'Ribeira de Pena',
                'Sabrosa',
                'Santa Marta de Penaguião',
                'Valpaços',
                'Vila Pouca de Aguiar',
                'Vila Real'
                ],
    'Viseu': [
                'Armamar',
                'Carregal do Sal',
                'Castro Daire',
                'Cinfães',
                'Lamego',
                'Mangualde',
                'Moimenta da Beira',
                'Mortágua',
                'Nelas',
                'Oliveira de Frades',
                'Penalva do Castelo',
                'Penedono',
                'Resende',
                'Santa Comba Dão',
                'São João da Pesqueira',
                'São Pedro do Sul',
                'Sátão',
                'Sernancelhe',
                'Tabuaço',
                'Tarouca',
                'Tondela',
                'Vila Nova de Paiva',
                'Viseu',
                'Vouzela'
                ]
}

In [11]:
# Exemplo de trabalho
user_distrito = 'Viseu'
user_concelho = 'Coimbra'

## SCRAPING

Foi construído o request tendo em conta a estrutura do site do Casa Sapo, que requer a informação do distrito e concelho do ponto anterior. O scraping foi feito com o módulo BeautifulSoup, que extrai os dados relevantes para o cálculo.

In [12]:
from unidecode import unidecode

In [23]:
url_distrito = unidecode(user_distrito.lower().replace(' ', '-'))
url_concelho = unidecode(user_concelho.lower().replace(' ', '-'))
url_req = 'https://casa.sapo.pt/comprar-apartamentos/' + url_concelho + '/'

In [14]:
url_req

'https://casa.sapo.pt/comprar-apartamentos/coimbra/'

A informação dos imóveis anunciados pode ser extraída com o Selenium e o BeautifulSoup:

In [2]:
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
def create_driver(proxy=None):

    options = webdriver.ChromeOptions()
    options.headless=True
    options.use_subprocess=True

    if proxy:
        options.add_argument('--proxy-server=%s' % proxy)
    
    return webdriver.Chrome(options=options)

In [6]:
driver = create_driver()
driver.get(url_req)
soup = BeautifulSoup(driver.page_source, "html.parser")
soup

<html lang="en"><head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/ch